In [ ]:
import os
import random
from tqdm import tqdm
import numpy as np
import torch, torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data.dataset import Dataset   

torch.backends.cudnn.benchmark=True

In [ ]:
##### Hyperparameters for federated learning #########

num_clients = 20

num_selected = 6

num_rounds = 150

epochs = 5

batch_size = 32


In [ ]:
# Image augmentation 

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Normalizing the test images

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

In [ ]:
##### Creating desired data distribution among clients  #####

# Loading CIFAR10 using torchvision.datasets
traindata = datasets.CIFAR10('./data', train=True, download=True, transform= transform_train)

# Dividing the training data into num_clients, with each client having equal number of images
traindata_split = torch.utils.data.random_split(traindata, [int(traindata.data.shape[0] / num_clients) for _ in range(num_clients)])

# Creating a pytorch loader for a Deep Learning model
train_loader = [torch.utils.data.DataLoader(x, batch_size=batch_size, shuffle=True) for x in traindata_split]

# Loading the test iamges and thus converting them into a test_loader
test_loader = torch.utils.data.DataLoader(
        datasets.CIFAR10('./data', train=False, transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
        ), batch_size=batch_size, shuffle=True)

Extracting ./data/cifar-10-python.tar.gz to ./data


In [ ]:
# Defining VGG16
import torchvision.models as models
vgg19 = models.vgg19_bn(pretrained=False)

In [ ]:
##### Federated Learning #####
# Client Local Training
def client_update(client_model, optimizer, train_loader, epoch=5):
    model.train()
    for e in range(epoch):
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.cuda(), target.cuda()
            optimizer.zero_grad()
            output = client_model(data)
            loss = F.cross_entropy(output, target)
            loss.backward()
            optimizer.step()
    return loss.item()

In [ ]:
##### Federated Learning #####
# Server aggregating
def server_aggregate(global_model, client_models):
      global_dict = global_model.state_dict()
      for k in global_dict.keys():
          global_dict[k] = torch.stack([client_models[i].state_dict()[k].float() for i in range(len(client_models))], 0).mean(0)
      global_model.load_state_dict(global_dict)
      for model in client_models:
          model.load_state_dict(global_model.state_dict())

In [ ]:
# Test
def test(global_model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.cuda(), target.cuda()
            output = global_model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    acc = correct / len(test_loader.dataset)

    return test_loss, acc

In [ ]:
#### Initializing models and optimizer  ####

# global model 
global_model = vgg19.cuda()

# client models 
client_models = [vgg19.cuda() for _ in range(num_selected)]

for model in client_models:
    model.load_state_dict(global_model.state_dict()) ### initial synchronizing with global model 

# optimizers
opt = [optim.SGD(model.parameters(), lr=0.01) for model in client_models]

In [ ]:
###### List containing info about learning #########
losses_train = []
losses_test = []
acc_train = []
acc_test = []
# Runnining FL

for r in range(num_rounds):
    # select random clients
    client_idx = np.random.permutation(num_clients)[:num_selected]
    # client update
    loss = 0
    for i in tqdm(range(num_selected)):
        loss += client_update(client_models[i], opt[i], train_loader[client_idx[i]], epoch=epochs)
    
    losses_train.append(loss)
    # server aggregate
    server_aggregate(global_model, client_models)
    
    test_loss, acc = test(global_model, test_loader)
    losses_test.append(test_loss)
    acc_test.append(acc)
    print('%d-th round' % r)
    print('average train loss %0.3g | test loss %0.3g | test acc: %0.3f' % (loss / num_selected, test_loss, acc))


  0%|          | 0/6 [00:00<?, ?it/s]

0-th round
average train loss 1.74 | test loss 1.68 | test acc: 0.470


  0%|          | 0/6 [00:00<?, ?it/s]

1-th round
average train loss 1.26 | test loss 2.23 | test acc: 0.425


  0%|          | 0/6 [00:00<?, ?it/s]

...

  0%|          | 0/6 [00:00<?, ?it/s]

135-th round
average train loss 0.0207 | test loss 0.482 | test acc: 0.904


  0%|          | 0/6 [00:00<?, ?it/s]

136-th round
average train loss 0.0342 | test loss 0.455 | test acc: 0.899


 33%|███▎      | 2/6 [01:23<02:47, 41.76s/it]

In [ ]:
# Centralized training
traindata = datasets.CIFAR10('./data', train=True, download=True, transform= transform_train)

# Creating a pytorch loader for a Deep Learning model
train_loader = torch.utils.data.DataLoader(traindata, batch_size=batch_size, shuffle=True)

vgg19 = models.vgg19_bn(pretrained=False)
vgg19.cuda()

opt = optim.SGD(vgg19.parameters(), lr=0.01)

losses_train = []
losses_test = []
acc_train = []
acc_test = []

for r in range(num_rounds):
    
    loss = client_update(vgg19, opt, train_loader, epoch=epochs)
    
    losses_train.append(loss)    
    test_loss, acc = test(vgg19, test_loader)
    losses_test.append(test_loss)
    acc_test.append(acc)
    print('%d-th round' % r)
    print('average train loss %0.3g | test loss %0.3g | test acc: %0.3f' % (loss / num_selected, test_loss, acc))


Files already downloaded and verified
0-th round
average train loss 0.152 | test loss 0.895 | test acc: 0.701
1-th round
average train loss 0.0738 | test loss 0.646 | test acc: 0.786
2-th round
average train loss 0.117 | test loss 0.618 | test acc: 0.802
3-th round
average train loss 0.129 | test loss 0.629 | test acc: 0.811
4-th round
average train loss 0.0197 | test loss 0.518 | test acc: 0.835
5-th round
average train loss 0.0377 | test loss 0.505 | test acc: 0.843
6-th round
average train loss 0.0477 | test loss 0.513 | test acc: 0.847
7-th round
average train loss 0.00885 | test loss 0.467 | test acc: 0.869
8-th round
average train loss 0.00954 | test loss 0.528 | test acc: 0.861
9-th round
average train loss 0.0173 | test loss 0.528 | test acc: 0.864
10-th round
average train loss 0.0363 | test loss 0.532 | test acc: 0.864
11-th round
average train loss 0.019 | test loss 0.565 | test acc: 0.865
12-th round
average train loss 0.0107 | test loss 0.545 | test acc: 0.872
13-th round
